In [74]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [75]:
# if using mac
df = pd.read_excel('/Users/zhanghantang/PycharmProjects/sucide_project/BIOM40forUSC.xlsx')
data = df[df['SI'].notnull()]
y = data['SI']
x = data.loc[:, 'GIMAP1Biom1552316_a_at':'CFIS']

In [71]:
# windows
df = pd.read_excel('E:\PHD\sucide_project\BIOM40forUSC.xlsx')
data = df[df['SI'].notnull()]
y = data['SI']
x = data.loc[:, 'GIMAP1Biom1552316_a_at':'CFIS']

FileNotFoundError: [Errno 2] No such file or directory: 'E:\\PHD\\sucide_project\\BIOM40forUSC.xlsx'

In [76]:
# column 'CFI-S.PheneVisit' data type is string, i am not clear its internal meaning and how to convert to float data type
x = x.drop(labels='CFI-S.PheneVisit', axis=1)
# drop these column directly
x = data.loc[:, 'GIMAP1Biom1552316_a_at':'RAB3GAP2Biom240234_at']
new_y = []
for i in y:
    if i == 0 or i ==1:
        new_y.append(0)
    else:
        new_y.append(1)
new_y = np.array(new_y)

In [77]:
# train test split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression as LR
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
# after test, standard scaler is neccessary
X = StandardScaler().fit_transform(x)
X_train, X_test, y_train, y_test = train_test_split(X, new_y, test_size=0.33, random_state=10)
print(len(X_train), len(X_test))


505 249


In [112]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression

# test parameter individually

# Create the K-Fold object
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Define your model
model = LR(solver='saga', penalty='elasticnet', C=100, max_iter=3000, l1_ratio=0.5)

# Use the cross_val_score function to obtain the scores for each fold
scores = cross_val_score(model, X, new_y, cv=kf, scoring='f1')


# Print the scores for each fold and the mean score
print("Scores: ", scores)
print("Mean Score: ", scores.mean())

'''
Scores:  [0.24390244 0.11764706 0.35897436 0.25531915 0.18181818]
Mean Score:  0.23153223751532614
'''

Scores:  [0.24390244 0.11764706 0.35897436 0.25531915 0.18181818]
Mean Score:  0.23153223751532614


'\nScores:  [0.24390244 0.11764706 0.35897436 0.25531915 0.18181818]\nMean Score:  0.23153223751532614\n'

In [108]:
# tune parameter
from sklearn.model_selection import GridSearchCV
model = LR(solver='saga')
# 不同的solver可能只支持不同的有限的penalty
param_grid = {'penalty':['l1', 'l2'], 'C':[0.1, 1, 10, 100], 'max_iter':[3000, 3500]}
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='f1', return_train_score=True)
grid_search.fit(X, new_y)
print("Best penalty: ", grid_search.best_params_['penalty'])
print("Best C: ", grid_search.best_params_['C'])
# print("solver: ", grid_search.best_params_['solver'])
print("Best score: ", grid_search.best_score_)

'''
Best penalty:  l1
Best C:  100
Best score:  0.19299478970021897

'''

Best penalty:  l1
Best C:  100
Best score:  0.19299478970021897


In [113]:
best_lr_model = LR(C=100, penalty='l1', solver='saga', max_iter=3000)
best_lr_model.fit(X_train, y_train)
y_pred = best_lr_model.predict(X_test)
print("Score with L1 penalty: accuracy:{} precision:{} recall:{} f1:{} roc_auc:{}".format(accuracy_score(y_test, y_pred), precision_score(y_test, y_pred), recall_score(y_test, y_pred), f1_score(y_test, y_pred), roc_auc_score(y_test, y_pred)))
'''
Score with L1 penalty: accuracy:0.7831325301204819 precision:0.4074074074074074 recall:0.22448979591836735 f1:0.2894736842105263 roc_auc:0.5722448979591838
'''

Score with L1 penalty: accuracy:0.7831325301204819 precision:0.4074074074074074 recall:0.22448979591836735 f1:0.2894736842105263 roc_auc:0.5722448979591838


In [ ]:
'''
-------------------------
the content above is about logistic regression content
-------------------------
'''